In [13]:
from pylab import *
import timeit
from scipy import optimize
import pandas as pd

# Problem 1:

In [14]:
def loop(C,r):
    PV = 0
    for i in range(len(C)):
        PV += C[i]/(1+r)**(i+1)
    return PV

def Horner(C,r):
    PV = 0
    for i in range(n-1,-1,-1):
        PV = PV/(1+r)+C[i]
    return PV/(1+r)
   
def Polyval(C,r):
    C1 = append(C[::-1],0)
    PV = polyval(C1, 1/(1+r))
    return PV

def dot_product(C,r):
    R = (1/(1+r))**arange(1,len(C)+1,1)
    PV = dot(C,R)
    return PV

C = 120.0 * arange(500,1200)
r = 0.01
n = len(C)

print("Using Loop:",loop(C,r))
print("Horner Scheme:",Horner(C,r))
print("Polyval:",Polyval(C,r))
print("Dot-Product",dot_product(C,r))

t = timeit.Timer('loop(C,r)', 'from __main__ import loop,C,r')
print("Run time for explicit loop = ", t.timeit(number=1000), "s")

t = timeit.Timer('Horner(C,r)', 'from __main__ import Horner,C,r')
print("Run time for Horner Scheme = ", t.timeit(number=1000), "s")

t = timeit.Timer('Polyval(C,r)', 'from __main__ import Polyval,C,r')
print("Run time using Polyval = ", t.timeit(number=1000), "s")

t = timeit.Timer('dot_product(C,r)', 'from __main__ import dot_product,C,r')
print("Run time using dot product = ", t.timeit(number=1000), "s")

Using Loop: 7185271.349713663
Horner Scheme: 7185271.3497136645
Polyval: 7185271.3497136645
Dot-Product 7185271.349713668
Run time for explicit loop =  0.27324916699990354 s
Run time for Horner Scheme =  0.17193383299991183 s
Run time using Polyval =  0.17449683299992103 s
Run time using dot product =  0.017487584000036804 s


# Problem 2:

First calculate the amount 'TA' such that amount 'C' be taken out for the next '$n_1$' years '$m_1$' times a year.
$$TA = C + \frac{C}{1+\frac{r}{m_1}} + \frac{C}{\left(1+\frac{r}{m_1}\right)^2} + \cdot \cdot \cdot + \frac{C}{\left(1+\frac{r}{m_1}\right)^{n_1m_1-1}} $$

Since amount 'TA' is reached in '$n_2$' years '$m_2$' times a year. 
$$TA = A\left({1+\frac{r}{m_2}}\right) + A\left({1+\frac{r}{m_2}}\right)^2 + \cdot \cdot \cdot + A\left({1+\frac{r}{m_2}}\right)^{n_2m_2} $$


In [15]:
C = 2000
r = 0.02

n1 = 30
m1 = 12
R = ((1+r/m1)**(-1))**(arange(0,n1*m1,1))
TA = C*sum(R)

n2 = 40
m2 = 12
R = (1+r/m2)**(arange(1,n2*m2+1,1))
A = TA/sum(R)
print(A)

736.7521601916105



# Problem 3:

Monthly installment:
\begin{equation}C = PV \cdot \frac{r}{m}\left( 1- \left( 1 + \frac{r}{m}\right)^{-nm} \right)^{-1} \tag{1}\end{equation}
Effective interest rate:
\begin{equation}r_{eff} = \left( 1 + \frac{r}{m} \right)^m - 1\tag{2}\end{equation}
Remaining principle after 'k' installments:
\begin{equation}PV_k = C \sum_{i = 1}^{nm-k}\left( 1+\frac{r}{m}\right)^{-i} \tag{3}\end{equation}
Principle paid in '$k^{th}$' installment:
\begin{equation} P_k = PV_{k-1} - PV_k \tag{4}\end{equation}
Interest paid in '$k^{th}$' installment:
\begin{equation} I_k = C - P_k  \tag{5}\end{equation}

In [17]:
def amortization(PV,r,m,n):
    C = PV*r/m*(1-(1+r/m)**(-n*m))**(-1)
    r_eff = (1+r/m)**m - 1
    
    #Create array whose elements are terms in the expansion of equation 3.
    R = ((1+r/m)**(-1))**arange(1,n*m+1,1)
    
    PVk = []
    PVk.append(C*sum(R[0:n*m-1]))
    
    P = []
    P.append(PV - PVk[0])
    
    for k in range(2,n*m+1):
        PVk.append(C*sum(R[0:n*m-k]))
        P.append(PVk[k-2] - PVk[k-1])
        
    I = zeros(len(P)) + C - P
    
    return C,r_eff, PVk, P, I
        

PV = 500000
r = 0.02
m = 12
n = 20

C,r_eff, PVk, P, I = amortization(PV,r,m,n)

print("Monthly payment = ", C)
print("Effective annual interest rate = ", r_eff)
print("\n")

dict = {'Remaining Principle' : PVk, 'Principle Paid' : P, 'Interest Paid' : I}
df = pd.DataFrame(dict)
df.style
df.index.name = 'k'
df.index += 1
print(df.to_string())

Monthly payment =  2529.4166752255383
Effective annual interest rate =  0.020184355681501787


     Remaining Principle  Principle Paid  Interest Paid
k                                                      
1          498303.916658     1696.083342     833.333333
2          496605.006511     1698.910147     830.506528
3          494903.264846     1701.741664     827.675011
4          493198.686946     1704.577900     824.838775
5          491491.268082     1707.418864     821.997812
6          489781.003520     1710.264562     819.152113
7          488067.888518     1713.115003     816.301673
8          486351.918323     1715.970194     813.446481
9          484633.088179     1718.830145     810.586531
10         482911.393317     1721.694862     807.721814
11         481186.828964     1724.564353     804.852322
12         479459.390337     1727.438627     801.978048
13         477729.072646     1730.317691     799.098984
14         475995.871092     1733.201554     796.215121
15       

# Problem 4:

$$NPV(r) = \sum_{i=1}^{n}\frac{c_i}{(1+r)^i} - P$$

In [20]:
n = 10
C = 120.0 * arange(42,52)
P = 50000.0

#Polynomial in x=1/(1+r)
def f(x):
    return dot(C, x**arange(1,n+1,1) ) - P

root = optimize.brentq(f, 0, 2)
print("r = ", 1/root-1)

r =  0.019803285095871592
